<a href="https://colab.research.google.com/github/MahdiJahanbakht/cat-dog-classification-on-colab/blob/master/4_Loading_checkpoints_and_Make_Inferences_or_Resume_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Previousely saved checkpoints
To do so we have to do as follows:
1.  Reconstruct the model from the structure saved in the checkpoint
2.  Load the state dict to the model  

Then, resume learning or...

3. Freeze the parameters and enter evaluation mode if you are loading the model for inference  

The following block does this for you

In [0]:
import torch, torchvision, os
from torch import nn, optim
from torchvision import models

filepath = '/content/drive/My Drive/Deep_Learning/Cat_Dog_run/checkpoint.pth'
for_eval = True

# Use Gpu if it's available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Download pre-trained model
model = models.resnet50(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Define cost function aas the negative log likelihood loss. It is useful to train a classification problem with C classes
criterion = nn.NLLLoss()

# Adam optimizer with learning rate of 0.003
optimizer = None

if os.path.isfile(filepath):
  
  checkpoint = torch.load(filepath,)
  start_epoch = checkpoint['epoch']
  
  model.fc = checkpoint['model']
  model.fc.load_state_dict(checkpoint['state_dict'])
  
  optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
  optimizer.load_state_dict(checkpoint['optimizer'])
  
  if for_eval == True:
    for parameter in model.fc.parameters():
        parameter.requires_grad = False
    model.eval()
  else:
    model.train()

  model.to(device)      
else:
  print("=> no checkpoint found at '{}'".format(filepath))
            



Next, you can run Learning block of previous notebook.
>**Caution:** If you are planning for resume learning, don't forget to update `start_epoch` variable by its new value to resume your work properly  
>**Note:** To just evalute the model you can use codes in the proceeding blocks

In [0]:
baseDir = '/content/drive/My Drive/Deep_Learning/'
dataDir = baseDir + 'Datasets/Cat_Dog_data/'
runDir = baseDir + 'Cat_Dog_run/'

# we used batch size of 64 images for batch learning
batch_size = 64

# For test data we did'nt use any form of Data Augmentation and just used
# resize and center crop to adjust our images for input
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# Define train and test folders located on google drive
testset = datasets.ImageFolder(dataDir+'test',transform=test_transforms)

# Define batch loaders
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)

In [0]:
eval_loss = 0
accuracy = 0
# Do not compute gradients. Speeds up the algorithm alot
with torch.no_grad():
    # Load test data
    for inputs, labels in testloader:
        # Move test tensors to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Do the forward pass
        logps = model.forward(inputs)
        # Compute cost
        batch_loss = criterion(logps, labels)
        # Sum up all the costs
        eval_loss += batch_loss.item()

        # Calculate accuracy. As you may remember we done LogSoftmax
        # in the last layer. So, to actual probabolities we should
        # calculate exp of output values
        ps = torch.exp(logps)
        # Select the best class
        top_p, top_class = ps.topk(1, dim=1)
        # Reshape our labels to match top_class shape
        equals = top_class == labels.view(*top_class.shape)
        # Change the type of equals to float tensor
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
# Keep track of test losses for each batch
mean_eval_loss = eval_loss/len(testloader)
mean_acc = accuracy/len(testloader)

print(f"Epoch {epoch+1}/{epochs}.. "
      f"Test loss: {mean_eval_loss:.3f}.. "
      f"Test accuracy: {mean_acc:.3f}")